In [223]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Pré-processamento #

### Base e encoder manual ###

In [224]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [225]:
warnings.filterwarnings("ignore")

In [226]:
df_encod_manual = pd.DataFrame.copy(df)

In [227]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [228]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [229]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [230]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [231]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [232]:
previsores_label = df.iloc[:, 0:11].values

In [233]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [234]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [235]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [236]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [237]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [238]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [239]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)

In [240]:
x_train.shape

(1405, 20)

In [241]:
x_test.shape

(603, 20)

In [242]:
y_train.shape

(1405,)

In [243]:
y_test.shape

(603,)

## Resumo do pré-processamento

<span style="font-size: 13px; font-family: 'Trebuchet MS', sans-serif;">

- alvo = variável que se pretende atingir (tem ou não doença cardíaca).</br> </br>
- previsores = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas manualmente, sem escalonar.</br></br>
- previsores_esc = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas, escalonada. </br></br>
- previsores_label = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas pelo labelencoder. </br></br>
- previsores_hot = conjunto de variáveis previsoras transformadas pelo labelencoder e onehotencoder, sem escalonar.</br></br>
- previsores_hot_esc = conjunto de variáveis previsoras transformadas pelo labelencoder e onehotencoder escalonada.</br></br>
</span>

# Classificação com redes neurais 
<span style="font-size: 13px; font-family: 'Trebuchet MS', sans-serif;">

As redes neurais artificiais (RNAs) são modelos computacionais inspirados no funcionamento do cérebro humano. </br> 
Elas são compostas por neurônios artificiais interconectados, organizados em camadas, e são capazes de aprender a partir de dados. </br>

**Estrutura das RNAs** </br>
Uma RNA é composta por várias camadas: </br>
- **Camada de Entrada:** Recebe os dados de entrada e transmite para as camadas ocultas. </br>
- **Camadas Ocultas:** Processam os dados de entrada através de uma série de transformações não-lineares. </br>
- **Camada de Saída:** Produz a saída final da rede após o processamento nas camadas ocultas. </br>

**Funcionamento das RNAs** </br>
As RNAs funcionam através de duas etapas principais: feedforward e backpropagation. </br>
- **Feedforward:** Durante esta etapa, os dados de entrada são propagados através da rede, camada por camada, até que a saída seja calculada. </br>
- **Backpropagation:** Durante esta etapa, o erro entre a saída prevista e a saída real é retropropagado pela rede, ajustando os pesos das conexões para minimizar o erro. </br>

**Treinamento das RNAs** </br>
- As RNAs são treinadas utilizando algoritmos de otimização, como o gradiente descendente, para ajustar os pesos das conexões de forma a minimizar uma função de custo ou perda.

</span>

## Criação do algoritmo

### Parâmetros MLPClassifier
<span style="font-size: 13px; font-family: 'Trebuchet MS', sans-serif;">

- **hidden_layer_sizes (camadas escondidas)**: default (100) </br>
- **NE** = Neuronios de ENTRADA </br>
- **NS** = Neuronios de SAIDA </br>

**FÓRMULA:**

- **Quant.**= $(NE+NS)/2$ = $(11+1)/2$ = $6$ neurônios </br>
- **Quant.**= $2/3.(NE)$ + $NS = 2/3.11+1$ = $8$ neurônios </br>
---
**activation**: Função de ativação default= 'relu' </br>
- **'identity'**: $f(x) = x$ </br>
Esta função retorna simplesmente a entrada, útil quando nenhuma ativação é desejada.
- **'logistic'**: $f(x) = \frac{1}{1 + e^{-x}}$ </br>
Também conhecida como função sigmóide, esta função transforma a entrada em um intervalo entre 0 e 1.

- **'tanh'**: $f(x) = \tanh(x)$ </br>
A função tangente hiperbólica, que mapeia a entrada para um intervalo entre -1 e 1.

- **'relu'**: $f(x) = \max(0, x)$ </br>
A função de ativação ReLU (Unidade Linear Retificada), que retorna 0 para valores negativos e a própria entrada para valores positivos.

---

**solver:** 
algoritmo matemático. Default='adam' </br>
- **adam** é para datasets grandes = acima de 1000 amostras
- **lbfgs** é para datasets pequenos.
- **sgd** é com a descida do gradiente estocástico (recomendado testar).
- **alpha**: parâmetro para o termo de regularização de ajuste de pesos.  
Aumento de alpha estimula pesos menores e diminuição de alpha estimula pesos maiores. Default=0.0001. </br>  

---
**batch_size: tamanho dos mini lotes.**
- default=min(200, n_samples). </br>
*Não usar com o solver lbfgs.*

---

**learning_rate:** taxa de aprendizagem. default='constant'. Três tipos:

- **'constant'**: uma taxa de aprendizado constante dada pela taxa de aprendizagem inicial.
- **'invscaling'**: diminui gradualmente por:  taxa efetiva = taxa inicial / $ t^power_t $
- **'adaptive'**: a taxa é dividida por 5 cada vez que em duas épocas consecutivas não diminuir o erro

---

**learning_rate_init**: taxa de aprendizagem inicial. 
- Default=0.001  

---

**max_iter int**: Número máximo de iterações. 
- default = 200.
('sgd', 'adam').  

---

**max_fun**: Número máximo de chamadas de função de perda.  
- Para 'lbfgs'. Default: 15000  

---

**shuffle**: default = True  
- Usado apenas quando solver = 'sgd' ou 'adam'.  

---

**random_state**: 
- default = None  

---

**tol**: Tolerância para a otimização. 
- Default=0.0001  

---

**momentum**: otimização do algoritmo 'sgd'. 
- Default: 0.9. 

---

**n_iter_no_change**: Número máximo de épocas que não atinge a tolerância de melhoria. default = 10.  
- Apenas para solver = 'sgd' ou 'adam'  

---

**verbose**: Mostra o progresso. 
- default=False.

</span>

In [244]:
redes = MLPClassifier(
    hidden_layer_sizes = (7, 7, 7), activation = 'relu',
    solver = 'adam', max_iter = 1000, alpha = 0.1,
    batch_size = 128, learning_rate = 'adaptive',
    learning_rate_init = 0.001,
    tol = 0.0001, random_state = 3, verbose = True
)

In [245]:
redes.fit(x_train, y_train);

Iteration 1, loss = 0.77171416
Iteration 2, loss = 0.75299658
Iteration 3, loss = 0.73646631
Iteration 4, loss = 0.72196969
Iteration 5, loss = 0.70970739
Iteration 6, loss = 0.69898574
Iteration 7, loss = 0.68935610
Iteration 8, loss = 0.68019868
Iteration 9, loss = 0.67112192
Iteration 10, loss = 0.66117823
Iteration 11, loss = 0.65043337
Iteration 12, loss = 0.63829468
Iteration 13, loss = 0.62472763
Iteration 14, loss = 0.61140656
Iteration 15, loss = 0.59756722
Iteration 16, loss = 0.58388369
Iteration 17, loss = 0.57178924
Iteration 18, loss = 0.56086005
Iteration 19, loss = 0.55062185
Iteration 20, loss = 0.54158110
Iteration 21, loss = 0.53371071
Iteration 22, loss = 0.52701105
Iteration 23, loss = 0.52039578
Iteration 24, loss = 0.51450930
Iteration 25, loss = 0.50924772
Iteration 26, loss = 0.50359208
Iteration 27, loss = 0.49837053
Iteration 28, loss = 0.49360359
Iteration 29, loss = 0.48914713
Iteration 30, loss = 0.48501683
Iteration 31, loss = 0.48105252
Iteration 32, los

In [246]:
previsoes = redes.predict(x_test)

### Métricas

In [247]:
print("Acurácia: %.2f%%" % (accuracy_score(y_test, previsoes) * 100.0))

Acurácia: 91.54%


In [248]:
print(confusion_matrix(y_test, previsoes))

[[317  21]
 [ 30 235]]


In [249]:
print(classification_report(y_test, previsoes))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       338
           1       0.92      0.89      0.90       265

    accuracy                           0.92       603
   macro avg       0.92      0.91      0.91       603
weighted avg       0.92      0.92      0.92       603



#### Métricas de treino

In [250]:
previsoes_train = redes.predict(x_train)

In [251]:
print("Acurácia em treino: %.2f%%" % (accuracy_score(y_train, previsoes_train) * 100.0))

Acurácia em treino: 93.67%


In [252]:
print(confusion_matrix(y_train, previsoes_train))

[[723  52]
 [ 37 593]]


In [253]:
print(classification_report(y_train, previsoes_train))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94       775
           1       0.92      0.94      0.93       630

    accuracy                           0.94      1405
   macro avg       0.94      0.94      0.94      1405
weighted avg       0.94      0.94      0.94      1405



### Cross Validation
<span style="font-size: 14px; font-family: 'Trebuchet MS', sans-serif;">

- Inicialmente, o conjunto de dados é dividido em k partes (ou folds) de aproximadamente o mesmo tamanho.
- O modelo é treinado k vezes. Em cada iteração, um dos k folds é retido como conjunto de teste e os outros k-1 folds são usados como conjunto de treinamento.
- Após completar as k iterações, a métrica de avaliação é calculada como a média dos resultados obtidos em cada fold de teste.

</span>

In [254]:
kfold = KFold(
    n_splits = 40, shuffle = True, random_state = 5
)

In [255]:
modelo = MLPClassifier(
    hidden_layer_sizes = (7, 7, 7), activation = 'relu',
    solver = 'adam', max_iter = 1000, alpha = 0.1,
    batch_size = 128, learning_rate = 'adaptive',
    learning_rate_init = 0.001,
    tol = 0.0001, random_state = 3, verbose = True
)
resultado = cross_val_score(
    modelo, previsoresHot_esc,
    alvo, cv = kfold
)

Iteration 1, loss = 0.76910293
Iteration 2, loss = 0.74298731
Iteration 3, loss = 0.72203986
Iteration 4, loss = 0.70500389
Iteration 5, loss = 0.69087150
Iteration 6, loss = 0.67748474
Iteration 7, loss = 0.66403591
Iteration 8, loss = 0.64853590
Iteration 9, loss = 0.63068369
Iteration 10, loss = 0.61144998
Iteration 11, loss = 0.59205833
Iteration 12, loss = 0.57411033
Iteration 13, loss = 0.55805269
Iteration 14, loss = 0.54391462
Iteration 15, loss = 0.53195939
Iteration 16, loss = 0.52155955
Iteration 17, loss = 0.51183093
Iteration 18, loss = 0.50353945
Iteration 19, loss = 0.49580537
Iteration 20, loss = 0.48891001
Iteration 21, loss = 0.48255948
Iteration 22, loss = 0.47686499
Iteration 23, loss = 0.47129571
Iteration 24, loss = 0.46609150
Iteration 25, loss = 0.46119777
Iteration 26, loss = 0.45667138
Iteration 27, loss = 0.45249465
Iteration 28, loss = 0.44831307
Iteration 29, loss = 0.44430196
Iteration 30, loss = 0.44058163
Iteration 31, loss = 0.43658009
Iteration 32, los

In [256]:
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 91.23%
